In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import pandas as pd
import PIL

from skimage import measure
from skimage.io import imread
from skimage.segmentation import clear_border

import ipywidgets as widgets

sys.path.append('../libraries')
from ring_functions import make_rings
import input_functions as inp_f

In [2]:
info_file_path = r'Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\info_B02.txt'

In [3]:
# read the info file
info_file = open(info_file_path, 'r')
info_lines = info_file.readlines()
info_file.close()

# read info about the data frame
exp_dir,df_name = inp_f.read_df_info(info_lines)

# get info about the channels
channel_list = inp_f.read_channels(info_lines)

# setting directories
labels_dir = os.path.join(exp_dir,'segmentation')
im_dir = os.path.join(exp_dir,'data')
df_dir = os.path.join(exp_dir,'df')

# reading labels 
file_list = [x for x in os.listdir(labels_dir) if 'label' in x]

In [4]:
# setting properties to calculate
properties = ['label', 'area','centroid','orientation','major_axis_length','minor_axis_length','bbox','image','mean_intensity']
properties_ring = ['label','centroid','mean_intensity']

In [5]:
# create slider
progress_bar = widgets.IntProgress(
    step=1,
    description='Processing:',
    orientation='horizontal',
    min = 0,
    max = len(file_list)-1,
    value = 0
)

In [6]:
#%%time

display(progress_bar)
cellDataList=[]

PIL.Image.MAX_IMAGE_PIXELS = None

for label_file in file_list:

    frame = int(label_file.split('_')[-2])
    print(frame)
    
    # update about progress
    progress_bar.value = frame
    
    # open labels
    label_path = os.path.join(labels_dir,label_file)
    labels_2D = imread(label_path)
    
    # clear border objects
    labels_2D = clear_border(labels_2D)
    
    # read images for intensity calculations
    intensity_list = []
    for i in np.arange(len(channel_list)):
        
        im_name = [ch['file_name'] for ch in channel_list if ch['channel_number']==i][0]
        ch_number = [ch['channel_in_file'] for ch in channel_list if ch['channel_number']==i][0]
        im_path = os.path.join(exp_dir,'data',im_name)
        print(im_path)
              
        im = inp_f.open_image(im_path,c=ch_number,t=frame)     
        print(im)
        
        intensity_list.append(im)

    int_im = np.moveaxis(np.array(intensity_list),0,2)

    # calculate properties of regions
    cellData = pd.DataFrame(measure.regionprops_table(labels_2D, properties=properties,intensity_image=int_im))

    # add info of these measurements
    cellData['file'] = label_file
    cellData['t'] = frame
    
    # calculate signals in rings
    rings = make_rings(labels_2D,width=6,gap=1)
    rings_prop = measure.regionprops_table(rings, properties=properties_ring,intensity_image=int_im)
    rings_prop = pd.DataFrame(rings_prop)

    cellData = pd.merge(cellData,rings_prop,how='inner',on='label',suffixes=('_nuc', '_ring'))

    cellDataList.append(cellData)

# put all together
cellDataAll = pd.concat(cellDataList,ignore_index=True)

# rename columns
cellDataAll.columns = ['label', 'area', 'centroid-0', 'centroid-1',
                       'orientation','major_axis_length', 
                       'minor_axis_length', 'bbox-0', 'bbox-1', 'bbox-2','bbox-3', 
                       'image'] + [f'mean_intensity-{x}_nuc' for x in np.arange(len(channel_list))]+['file', 
                        't', 'centroid-0_ring','centroid-1_ring'] + [f'mean_intensity-{x}_ring' 
                                                                     for x in np.arange(len(channel_list))]


# add info
cellDataAll['size_x'] = labels_2D.shape[0]
cellDataAll['size_y'] = labels_2D.shape[1]

# save calculations
cellDataAll.to_pickle(os.path.join(df_dir,df_name),protocol = 4)
cellDataAll.to_csv(os.path.join(df_dir,df_name.replace('pkl','csv')),index=False)

IntProgress(value=0, description='Processing:', max=187)

0


C:\Users\gases\.conda\envs\livecell_GPU_segment\lib\site-packages\imageio\plugins\pillow.py:301: UserWarning: Loading 16-bit (uint16) PNG as int32 due to limitations in pillow's PNG decoder. This will be fixed in a future version of pillow which will make this warning dissapear.
  UserWarning,


Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[430 431 430 ... 394 393 392]
 [431 432 431 ... 394 394 393]
 [432 432 431 ... 395 394 393]
 ...
 [494 496 498 ... 425 425 424]
 [494 495 496 ... 424 423 421]
 [493 494 496 ... 422 421 421]]
1
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[392 393 391 ... 358 357 356]
 [392 393 392 ... 359 359 358]
 [392 392 391 ... 360 359 359]
 ...
 [470 471 472 ... 300 299 298]
 [470 471 472 ... 297 297 295]
 [471 472 472 ... 295 294 293]]
2
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[396 396 395 ... 341 341 340]
 [396 397 396 ... 341 341 341]
 [396 397 396 ... 342 342 341]
 ...
 [452 453 453 ... 258 255 253]
 [451 452 452 ... 255 253 251]
 [450 451 451 ... 253 251 250]]
3
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[380 381 380 ... 340 340 339]
 [381 382 381 ... 341 340 339]


29
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[374 375 374 ... 337 336 335]
 [375 376 376 ... 337 336 335]
 [376 377 376 ... 338 337 336]
 ...
 [445 447 448 ... 235 233 232]
 [445 446 448 ... 233 231 230]
 [444 446 447 ... 231 230 229]]
30
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[385 386 384 ... 338 337 337]
 [385 386 385 ... 338 337 337]
 [386 386 385 ... 338 338 337]
 ...
 [445 447 448 ... 237 236 234]
 [444 446 447 ... 236 234 232]
 [442 444 445 ... 234 232 230]]
31
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[391 391 390 ... 334 333 332]
 [391 391 391 ... 334 333 333]
 [391 392 391 ... 335 334 333]
 ...
 [446 446 446 ... 238 236 235]
 [445 445 445 ... 236 235 233]
 [444 444 445 ... 234 232 230]]
32
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[388 389 388 ... 335 333 332]
 [388 388 388 ... 335 334

58
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[425 425 424 ... 333 332 332]
 [425 426 425 ... 333 332 332]
 [426 427 426 ... 333 332 332]
 ...
 [466 466 466 ... 231 230 228]
 [465 465 465 ... 229 228 227]
 [463 464 464 ... 228 227 226]]
59
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[427 428 427 ... 337 336 334]
 [427 428 427 ... 338 336 335]
 [428 429 427 ... 339 338 336]
 ...
 [462 462 462 ... 233 232 230]
 [461 461 461 ... 231 230 227]
 [461 460 461 ... 229 227 225]]
60
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[418 418 417 ... 346 345 346]
 [419 419 418 ... 347 347 346]
 [419 419 418 ... 348 348 347]
 ...
 [458 458 458 ... 228 225 223]
 [456 456 456 ... 225 222 219]
 [455 455 456 ... 222 219 216]]
61
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[411 412 411 ... 345 344 343]
 [411 412 412 ... 345 345

87
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[397 396 395 ... 337 336 336]
 [396 396 395 ... 337 337 337]
 [396 396 395 ... 338 338 337]
 ...
 [431 432 433 ... 225 223 221]
 [430 432 433 ... 223 220 219]
 [430 431 432 ... 220 218 216]]
88
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[397 397 396 ... 332 331 330]
 [397 397 397 ... 332 330 329]
 [397 397 396 ... 333 331 330]
 ...
 [433 433 434 ... 226 224 222]
 [433 433 434 ... 224 222 220]
 [433 434 434 ... 221 220 219]]
89
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[394 394 394 ... 332 331 332]
 [395 395 395 ... 333 332 332]
 [396 396 396 ... 333 333 332]
 ...
 [432 432 433 ... 224 222 220]
 [431 432 432 ... 222 221 218]
 [431 431 432 ... 220 219 217]]
90
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[414 415 414 ... 334 332 331]
 [414 416 415 ... 333 332

116
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[386 387 385 ... 345 344 344]
 [387 387 386 ... 346 344 344]
 [388 388 386 ... 346 345 344]
 ...
 [439 439 440 ... 225 224 223]
 [439 439 439 ... 223 222 221]
 [439 439 439 ... 221 220 219]]
117
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[385 386 385 ... 344 343 342]
 [385 386 385 ... 344 343 342]
 [386 386 385 ... 344 343 342]
 ...
 [440 440 441 ... 214 211 209]
 [440 440 441 ... 212 209 207]
 [440 441 441 ... 210 208 205]]
118
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[383 383 382 ... 345 345 344]
 [383 383 383 ... 346 345 345]
 [383 384 383 ... 346 345 345]
 ...
 [434 435 435 ... 218 215 213]
 [433 433 434 ... 216 213 211]
 [432 432 433 ... 214 211 208]]
119
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[377 378 378 ... 347 346 346]
 [378 379 379 ... 348

145
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[388 388 386 ... 344 343 343]
 [388 388 386 ... 347 345 345]
 [388 388 386 ... 347 346 346]
 ...
 [454 454 455 ... 218 217 217]
 [453 453 453 ... 216 215 215]
 [453 453 454 ... 215 214 213]]
146
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[378 379 379 ... 341 340 338]
 [378 380 380 ... 342 341 340]
 [379 380 380 ... 343 341 340]
 ...
 [460 460 461 ... 224 223 222]
 [461 461 461 ... 221 220 219]
 [461 461 461 ... 219 218 218]]
147
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[381 381 381 ... 338 336 335]
 [381 381 382 ... 339 338 336]
 [381 382 381 ... 340 339 337]
 ...
 [460 460 460 ... 227 226 224]
 [461 460 460 ... 225 223 222]
 [460 460 461 ... 222 221 219]]
148
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[377 378 378 ... 342 341 340]
 [378 380 380 ... 342

174
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[401 401 400 ... 352 351 351]
 [401 401 400 ... 353 352 352]
 [400 401 400 ... 353 352 352]
 ...
 [447 447 447 ... 221 220 219]
 [447 447 447 ... 220 219 218]
 [446 447 448 ... 219 218 217]]
175
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[389 390 389 ... 348 347 348]
 [390 391 391 ... 349 348 348]
 [392 392 391 ... 350 349 349]
 ...
 [446 447 447 ... 218 215 212]
 [447 447 447 ... 215 212 209]
 [447 447 447 ... 211 209 206]]
176
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[387 387 386 ... 348 347 347]
 [387 387 387 ... 349 348 348]
 [387 387 387 ... 349 349 348]
 ...
 [446 446 447 ... 220 218 215]
 [446 446 447 ... 218 215 213]
 [446 446 447 ... 215 213 210]]
177
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\data\WellB02_ChannelCFP_Seq0000 - Stitched.nd2
[[383 384 384 ... 344 343 341]
 [383 385 385 ... 344